# RECIST v1.1 Timepoint

> Calculation of RECIST v1.1 Timepoint

In [ ]:
#| default_exp timepoint

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass, field
from typing import Dict
from recist_calc.target import RecistTarget
from recist_calc.nontarget import RecistNonTarget

## `RecistTimePoint` DataClass

### Definition

In [ ]:
#| export
@dataclass
class RecistTimePoint:
    """This class represents the RECIST (Response Evaluation Criteria in Solid Tumors) evaluation 
    for a specific time point, combining the assessments of target and non-target lesions.
    """
    target: RecistTarget = field(default_factory=RecistTarget)
    nontarget: RecistNonTarget = field(default_factory=RecistNonTarget)
    is_new_lesion: bool = field(default= None)
    category: Dict[str, str] = field(default_factory=dict, init=False)
    
    def __post_init__(self):
        # Assign Categories
        self.category = {"final": "", "target": self.target.category, "nontarget": self.nontarget.category}
        # Override New Lesion
        self.is_new_lesion = self.target.is_new_lesion or self.nontarget.is_new_lesion 
        # PD
        if self.target.category == "PD" or self.nontarget.category == "PD" or self.is_new_lesion:
            self.category["final"] = "PD"
        # CR
        elif self.target.category == "CR" and self.nontarget.category == "CR":
            self.category["final"] = "CR"
            
        # PR
        elif self.target.category == "CR" and self.nontarget.category in ["Non-CR/Non-PD", "NE"]:
            self.category["final"] = "PR"
        elif self.target.category == "PR" and self.nontarget.category == "Non-CR/Non-PD":
            self.category["final"] = "PR"
        # SD
        elif self.target.category == "SD" and self.nontarget.category == "Non-CR/Non-PD":
            self.category["final"] = "SD"
        # NE
        elif self.target.category == "NE":
            self.category["final"] = "NE"
        else:
            # self.category["final"] = "-"
            raise NotImplementedError


### Example

#### Progressive Disease (PD) - Due to New Lesion

**Scenario:** A new lesion has appeared, which indicates Progressive Disease (PD).

In [ ]:
target = RecistTarget(current_mm=25, baseline_mm=20, is_new_lesion=True)
nontarget = RecistNonTarget(unequivocal_progress=False)
timepoint = RecistTimePoint(target=target, nontarget=nontarget)

print(timepoint.category)

{'final': 'PD', 'target': 'PD', 'nontarget': 'Non-CR/Non-PD'}


#### Progressive Disease (PD) - Due to Increase from Nadir


**Scenario:** The target lesion has increased in size by more than 20% from the nadir with an absolute increase of at least 5 mm, indicating PD.

In [ ]:
target = RecistTarget(current_mm=40, baseline_mm=30, nadir_mm=30)
nontarget = RecistNonTarget(unequivocal_progress=False)
timepoint = RecistTimePoint(target=target, nontarget=nontarget)
print(timepoint.category)

{'final': 'PD', 'target': 'PD', 'nontarget': 'Non-CR/Non-PD'}


#### Complete Response (CR)

**Scenario:** Both target and non-target lesions have disappeared, indicating Complete Response (CR).

In [ ]:
target = RecistTarget(current_mm=0, baseline_mm=25, is_ln_pathological=False)
nontarget = RecistNonTarget(disappear_all=True, is_tumor_mk_normalized=True, is_ln_pathological=False)
timepoint = RecistTimePoint(target=target, nontarget=nontarget)
print(timepoint.category)  

{'final': 'CR', 'target': 'CR', 'nontarget': 'CR'}


#### Partial Response (PR)

**Scenario:** The target lesion has decreased in size by at least 30% from baseline, and the non-target lesion is stable, indicating Partial Response (PR).

In [ ]:
target = RecistTarget(current_mm=15, baseline_mm=30)
nontarget = RecistNonTarget(unequivocal_progress=False)
timepoint = RecistTimePoint(target=target, nontarget=nontarget)
print(timepoint.category) 

{'final': 'PR', 'target': 'PR', 'nontarget': 'Non-CR/Non-PD'}


#### Stable Disease (SD)

**Scenario:** The target lesion shows a minor decrease in size (less than 30% from baseline), and the non-target lesion is stable, indicating Stable Disease (SD).

In [ ]:
target = RecistTarget(current_mm=28, baseline_mm=30)
nontarget = RecistNonTarget(unequivocal_progress=False)
timepoint = RecistTimePoint(target=target, nontarget=nontarget)
print(timepoint.category)

{'final': 'SD', 'target': 'SD', 'nontarget': 'Non-CR/Non-PD'}


#### Not Evaluable (NE)

**Scenario:** Insufficient data is available to evaluate the target lesion, so the time point is categorized as Not Evaluable (NE).

In [ ]:
target = RecistTarget(current_mm=None, baseline_mm=30)
nontarget = RecistNonTarget(unequivocal_progress=None)
timepoint = RecistTimePoint(target=target, nontarget=nontarget)
print(timepoint.category)  


{'final': 'NE', 'target': 'NE', 'nontarget': 'NE'}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()